# 04: Identifying Power Outages Using Social Media - Mapping Outages
### Danielle Medellin, Matthew Malone, Omar Smiley

## Import Libraries 

In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# install bokeh if needed
#!pip install bokeh

from os import path
from PIL import Image
import bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, CARTODBPOSITRON_RETINA
from bokeh.io import output_file, show, output_notebook

from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.layouts import column, row

## Load Data

In [210]:
tweets = pd.read_csv('../data/cleaned_tweets.csv')

In [211]:
tweets['tweet_date'] = pd.to_datetime(tweets['tweet_date'])

## Bokeh Mapping

In [212]:
map_tweets = tweets.copy()

#### Converting Longitude and Latitude to Mercator Projection

In [213]:
# references for the below code from Noah Christiansen &
# https://wiki.openstreetmap.org/wiki/Mercator#Elliptical_.28true.29_Mercator_Projection


def convert_longitude(long):
    k = 6378137 # earth radius in meters 
    x = long * (k * np.pi/180)
    
    return x

In [214]:
# references for the below code from Noah Christiansen &
# https://wiki.openstreetmap.org/wiki/Mercator#Elliptical_.28true.29_Mercator_Projection

def convert_latitude(lat):
    k = 6378137 # earth radius in meters 
    y = np.log(np.tan((90 + lat) * np.pi/360)) * k
    
    return y

In [215]:
def convert_radius(radius):
    return int(radius[:-2])*1609.34 # convert miles to meters 

In [216]:
# convert lat and long to mercator 

map_tweets['lat'] = map_tweets['lat'].map(convert_latitude)
map_tweets['long'] = map_tweets['long'].map(convert_longitude)
map_tweets['radius'] = map_tweets['radius'].map(convert_radius)

In [217]:
def get_year(date):
    return date.year

In [218]:
def get_month(date):
    return date.month

In [219]:
def get_day(date):
    return date.day

In [220]:
map_tweets['year'] = map_tweets['tweet_date'].map(get_year)
map_tweets['month'] = map_tweets['tweet_date'].map(get_month)
map_tweets['day'] = map_tweets['tweet_date'].map(get_day)

In [222]:
map_tweets['tweet_date'][0].year

2016

In [224]:
map_tweets['year'].value_counts()

2019    7359
2020    4257
2017    1874
2018    1762
2016    1661
Name: year, dtype: int64

In [225]:
map_tweets.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start,name_and_tweet,outage_sentiment,state,year,month,day
0,710245730590404608,ttwn sf bay area,power outage in cupertino bayarea traffic,2016-03-16 23:25:52,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,ttwn sf bay area power outage in cupertino bay...,0.33,California,2016,3,16
1,708811502241734656,san jose now,weather alert flash flood watch in bay area am...,2016-03-13 00:26:45,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now weather alert flash flood watch i...,0.25,California,2016,3,13
2,706856719733776384,san jose now,power outages in san francisco on peninsula in...,2016-03-07 14:59:09,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now power outages in san francisco on...,0.18,California,2016,3,7
3,726876023573204993,san jose now,power outage in fremont several intersections ...,2016-05-01 20:48:43,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now power outage in fremont several i...,0.23,California,2016,5,1
4,724681945095888897,san jose now,east bay power outages also affects bart uc be...,2016-04-25 19:30:14,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now east bay power outages also affec...,0.20,California,2016,4,25


In [226]:
# separate data 

high_os = map_tweets[map_tweets['outage_sentiment'] >= .3]
low_os = map_tweets[map_tweets['outage_sentiment'] < .3]

In [229]:
# https://docs.bokeh.org/en/latest/docs/user_guide/geo.html

# base map
tile_provider = get_provider(CARTODBPOSITRON_RETINA)

source = ColumnDataSource(
    data=dict(lat=high_os['lat'], # converted latitudes
              long=high_os['long'], # converted longitudes 
              radius=high_os['radius'], # converted radius in meters
              year=high_os['year'],
              month=high_os['month'],
              day=high_os['day'],
              city = high_os['city'])
)

# Sidebar tools for the bokeh map
TOOLS="hover,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select"

TOOLTIPS =[
    ('City', '@city'),
    ('Month','@month'),
    ('Year','@year'),
    ('Day','@day')
]

callback = CustomJS(args=dict(source=source), code="""
    var source_data = source.data;
    var day = day.value;
    var month = month.value;
    var year = year.value;
    
    source_data.lat = []
    source_data.long = []
    source_data.radius = []
    source_data.city = []
    source_data.day = []
    source_data.month = []
    source_data.year = []
    
    for(var i =0; i < source_data.lat.length; i++) {
        if ((source_data.year[i] == year)&(source_data.month[i]==month)&(source_data.day[i]==day)) {
            source_data.lat.push(source_data.lat[i]);
            source_data.long.push(source_data.long[i]);
            source_data.radius.push(source_data.radius[i]);
            source_data.city.push(source_data.city[i])
            source_data.day.push(source_data.day[i])
            source_data.month.push(source_data.month[i])
            source_data.year.push(source_data.year[i])
        }
    }
    
    source.change.emit();
""")

# Slider to control day of the year data is shown for
day_slider = Slider(start=1,end=31, value=1, step=1, title="Day", callback = callback)
callback.args['day'] = day_slider

# Slider to control month of the year data is shown for
month_slider = Slider(start=1,end=12, value=1, step=1, title="Month", callback = callback)
callback.args['month'] = month_slider

# Slider to control year data is shown for
year_slider = Slider(start=2016, end=2020, value=2016, step=1,title="Year", callback = callback)
callback.args['year'] = year_slider


# range bounds supplied in web mercator coordinates
p = figure(x_range=(-15000000, -6900000), y_range=(2000000, 6500000), # shows US
           x_axis_type="mercator", y_axis_type="mercator", title='Power Outages in the United States (Tracked by Tweets)',
           tools = TOOLS, tooltips = TOOLTIPS)


p.add_tile(tile_provider)

p.circle(x="long", y="lat", size = 5, fill_color="yellow", line_color = 'red',line_width=2.5, fill_alpha=0.0, source=source, legend_label='Power Outage', radius="radius")


layout = column(year_slider, month_slider, day_slider, p)
curdoc().add_root(layout)
output_file("outage_map.html")
output_notebook()
show(layout)  # open a browser

#show(p)

AttributeError: unexpected attribute 'callback' to Slider, similar attributes are js_event_callbacks

## Adding Slider !